# q-analogues of Oscillating Tableaux

We wish to investigate possible q-statistics for q-analogues of enumerative formulas for oscillating tableaux.

#### Definition
Let $\lambda, \mu$ be straight shapes.	An *$n$-oscillating tableau* of shape $\lambda/\mu$ is a sequence
$ \mu = \nu^0, \nu^1, \nu^2, \ldots, \lambda $ of partitions such that for each $i$,

- $\nu^i$ differs from $\nu^{i-1}$ by a single box.

- $\ell(\nu^i) \leq n$.

In the literature this is also known as an *$n$-symplectic up-down tableau*. When the length restriction is implicit or not imposed, we will drop the $n$ and simply refer to this as an oscillating tableau or an up-down tableau.

Let $\widetilde{f}_\lambda^{k}$ denote the number of oscillating tableaux of shape $\lambda$ in $k$ steps.  By bijecting matchings on $2k$ elements to oscillating tableaux of shape $\emptyset$, one gets the following enumerative formulas:

$$ \widetilde{f}_\emptyset^{2k} = (2k-1)!! $$
$$ \sum_{\lambda} \widetilde{f}_\lambda^{k} = (2k-1)!! $$

Using Berele insertion, Sundaram proves a more general result:
$$ \widetilde{f}_\lambda^k = \binom{k}{|\lambda|}(k-|\lambda|-1)!! f^\lambda $$

There is a well known q-analogue of $f^\lambda$ related to the *principal specialization* of Schur functions:

$$ [f^\lambda]_q = q^{n(\lambda)} \frac{n!_q}{\prod_{x \in \lambda} h(x)_q} = \sum_{T \in SYT(\lambda)} q^{\text{maj}(T)} $$
$$ s_\lambda(1, q, q^2, \ldots) = q^{n(\lambda)} \frac{1}{\prod_{x \in \lambda} 1-q^{h(x)}} = \frac{[f^\lambda]_q}{(1-q) \cdots (1-q^n)} =\frac{\sum_{T} q^{\text{maj}(T)}}{(1-q)\cdots(1-q^n)}$$ 

where $n(\lambda) = \sum \binom{\lambda_i'}{2} = \sum (i-1) \lambda_i$ is the sum of the entries of the tableau of shape $\lambda$ in which you fill the first row with 0's, the second row with 1's, and so forth. The major statistic $\text{maj}(T)$ is given by
$$ \text{maj}(T) = \sum_{i \in \text{Des}(T)} i $$
where we say $i$ is a *descent* of $T$ if $i$ occurs in a row below $i+1$ (in French notation). 

#### Example:

In [4]:
Tableaux.options.convention="french"
t = StandardTableau([[1,2,4],[3,6],[5]])
t.pp()
print "Descents: ", t.standard_descents()

  5
  3  6
  1  2  4
 Descents:  [2, 4]


Representation theoretically, $ s_\lambda(1, q, q^2, \ldots)$ keeps track of the occurences of the irreducible representation $\chi_\lambda$ of $S_n$ indexed by $\lambda$ in the symmetric algebra $S^\bullet(\mathbb{C}^n)$. That is, $S_n$ acts on $\mathbb{C}^n$, and considering this as an algebraic variety, also acts on polynomial functions on $\mathbb{C}^n$. This algebra of polynomial functions is $S^\bullet(\mathbb{C}^n) \simeq \mathbb{C}[x_1, \ldots, x_n]$ after picking a basis. This representation is graded by total degree, as $S_n$ doesn't change the total degree of a monomial in the $x$'s. If you want to know in which degrees, and with what multiplicities, an irrep $\chi_\lambda$ appears, then this is given by $s_\lambda(1, q, q^2, \ldots)$, where $q$ keeps track of the degrees. For example, we see that $q^{n(\lambda)}$ is the smallest degree in which $\chi_\lambda$ appears, and it occurs once here. The entire Frobenius character of $S^\bullet(\mathbb{C}^n)$ is given by 

$$ F\chi_{S^\bullet(\mathbb{C}^n)} = \sum q^d F\chi_{S^d(\mathbb{C}^n)} = h_n\left[\frac{X}{1-q}\right] = h_n(x_1, x_2, \ldots, x_1 q, x_2 q, \ldots, x_1 q^2, x_2 q^2, \ldots) $$

In any case, descents for oscillating tableaux were defined by Rubey, Sagan, Westbury in https://pdfs.semanticscholar.org/ec2c/194f00ab693c3e9c027c50e7165b2c8c90a7.pdf

## Code

In [ ]:
S.<q> = QQ[]
from sage.combinat.q_analogues import *

def OscillatingTableaux(inner, outer, steps):
    ''' Return list of oscillating tableaux from 
        shape ``inner`` to shape ``outer`` in ``steps`` number of steps.
        
        Input:
            ``inner`` -- Partition or list
            ``outer`` -- Partition or list
            ``steps`` -- non-negative integer
    '''
    inner = Partition(inner)
    outer = Partition(outer)
    if abs(outer.size() - inner.size()) > steps or steps <= 0:
        return []
    if steps == 1:
        if outer in inner.up_list() or outer in inner.down_list():
            return [(inner, outer)]
        else:
            return []
    next_parts = inner.up_list() + inner.down_list()
    OTs = []
    for seq in next_parts:
        for ot in OscillatingTableaux(seq, outer, steps-1):
            OTs += [(inner,) + ot]
    return OTs

def descents(ot):
    ''' Return list of descents of oscillating tableau ``ot``.
    
        Input:
            ``ot`` -- list of partitions or lists.
    '''
    des = []
    steps = len(ot)
    for i in range(1,steps-1):
        prev = Partition(ot[i-1])
        curr = Partition(ot[i])
        nxt = Partition(ot[i+1])
        if prev.size() < curr.size() and curr.size() > nxt.size():
            des += [i]
        elif prev.size() < curr.size() and curr.size() < nxt.size():
            row_prev = SkewPartition([curr,prev]).cells()[0][0]
            row_curr = SkewPartition([nxt,curr]).cells()[0][0]
            if row_prev < row_curr:
                des += [i]
        elif prev.size() > curr.size() and curr.size() > nxt.size():
            row_prev = SkewPartition([prev, curr]).cells()[0][0]
            row_curr = SkewPartition([curr, nxt]).cells()[0][0]
            if row_prev > row_curr:
                des += [i]
    return des

def maj(ot):
    ''' Return maj of oscillating tableau ``ot``.    
    '''
    return sum(descents(ot))

def q_oscillating(inner, outer, steps):
    ''' Return q descent generating function of 
        oscillating tableaux of shape ``outer``/``inner`` in ``steps`` steps.    
    '''
    gen_fn = 0
    for ot in OscillatingTableaux(inner, outer, steps):
        gen_fn += q^(maj(ot))
    return gen_fn
   
def q_hook_formula(shape):
    ''' Return q-analogue of hook length formula for SYT.
    '''
    res = 0
    for st in StandardTableaux(shape):
        res += q^(st.standard_major_index())
    return res

def q_double_factorial(n):
    ''' Return q-analogue of n!!
        
        The q-analogue is assumed to be the product of the q-analogues of
        numbers less than or equal to n, with the same parity as n. 
        
    '''
    res = 1
    if n % 2 == 0:
        start = 0
    else:
        start = 1
    for i in range(start,n+1,2):
        res = res*q_int(i)
    return res

def q_oscillating_formula(shape, steps):
    size = Partition(shape).size()
    return q_binomial(steps, size)*q_double_factorial(steps-size-1)*q_hook_formula(shape)

## Testing

In [8]:
# collect q-maj generating functions for oscillating tableaux and print them

res = {}
for size in range(5):
    for shape in Partitions(size):
        res[shape] = {}
        for steps in range(8):
            p = q_oscillating([], shape, steps)/q_hook_formula(shape)
            if steps > size:
                p = p/q_binomial(steps, size)
            if p != 0:
                res[shape][steps] = p.factor()
            else:
                res[shape][steps] = p

print "res: "
for k in res.keys():
    print "----------\n", "shape: ", k, "\n-----------"
    for s in res[k]:
        print "steps: ", s, " -- ", res[k][s]

res: 
----------
shape:  [2, 1, 1] 
-----------
steps:  0  --  0
steps:  1  --  0
steps:  2  --  0
steps:  3  --  0
steps:  4  --  1
steps:  5  --  0
steps:  6  --  q
steps:  7  --  0
----------
shape:  [1] 
-----------
steps:  0  --  0
steps:  1  --  1
steps:  2  --  0
steps:  3  --  q
steps:  4  --  0
steps:  5  --  q^2 * (q^2 - q + 1) * (q^2 + q + 1)
steps:  6  --  0
steps:  7  --  q^3 * (q^4 + q^3 + q^2 + q + 1) * (q^8 - q^7 + q^6 + q^4 + q^2 - q + 1)
----------
shape:  [2] 
-----------
steps:  0  --  0
steps:  1  --  0
steps:  2  --  1
steps:  3  --  0
steps:  4  --  q
steps:  5  --  0
steps:  6  --  q^2 * (q^2 - q + 1) * (q^2 + q + 1)
steps:  7  --  0
----------
shape:  [3, 1] 
-----------
steps:  0  --  0
steps:  1  --  0
steps:  2  --  0
steps:  3  --  0
steps:  4  --  1
steps:  5  --  0
steps:  6  --  q
steps:  7  --  0
----------
shape:  [3] 
-----------
steps:  0  --  0
steps:  1  --  0
steps:  2  --  0
steps:  3  --  1
steps:  4  --  0
steps:  5  --  q
steps:  6  --  0
step

# Feel free to mess around here

### NOTE: Factoring may not be the best, because we get negative signs. For example, factoring 1+q^3 gives (1-q+q^2)(1+q)